In [ ]:
import pandas as pd
import os
from tqdm import tqdm

import sys
sys.path.append('../')

from jem.model import jem
from jem.analyse import analyse
from jem.utils import *

In [ ]:
# #--------------------------------------------
# # Get damaged nodes

# node_path = '../data/risk-results/direct_damages/electricity_network_v1.0_nodes/'

# collected = []
# for f in tqdm(os.listdir(node_path)):
#     if 'csv' in f:
#         data = pd.read_csv(node_path + f)
#         collected.append(data)
        

# collected = pd.concat(collected,ignore_index=True)
# nodes_damaged = collected.id.unique()

In [ ]:
nodes = gpd.read_file('../data/spatial/infrasim-network/nodes.shp')
nodes_to_attack = nodes.loc[nodes.asset_type == 'junction'].head(5).id.to_list()

In [ ]:
#--------------------------------------------
# Simulate damages in loop

# define input files
path_to_flows = '../data/csv/generated_nodal_flows.csv'
path_to_nodes = '../data/spatial/infrasim-network/nodes.shp'
path_to_edges = '../data/spatial/infrasim-network/edges.shp'

n = []
x = []
y = []

nodes_to_attack = ['node_23715']

count = 0
for i in tqdm(range(0,len(nodes_to_attack))):
    if count < 99999999999999:
        # print(count)
        # init model
        run = jem(path_to_nodes,
                path_to_edges,
                path_to_flows,
                #timesteps=1,
                print_to_console=False,
                nodes_to_attack=list(nodes_to_attack[i]),
                #edges_to_attack=['edge_25297'],
                super_source=True,
                super_sink=False)
        
        # build model
        run.build()
        
        # run model
        run.optimise(print_to_console=False)
        
        # init results
        results = analyse(model_run=run)
        
        # get results
        try:
            n.append(nodes_damaged[0:i][-1])
        except:
            pass
        x.append(count)
        y.append(results.customers_affected_total())

    count = count + 1

r = pd.DataFrame({'x' : x, 'y' : y})

In [ ]:
results.nodes_with_shortfall()

In [ ]:
results.edge_flows[results.edge_flows.to_id=='node_23716']

In [ ]:
flows = pd.read_csv('../data/csv/generated_nodal_flows.csv')
flows

In [ ]:
def process_raw_consumption_data(path):
    '''Process raw consumption data from JPS
    '''
    elec_data  = pd.read_csv(path)
    elec_data  = elec_data[['PARISH','Year','PARISH LOAD']]
    # pivot data
    elec_data  = elec_data.pivot(index='PARISH',columns='Year')
    elec_data.columns = [str(i[1]) for i in elec_data.columns.values]
    elec_data = elec_data.reset_index()
    # convert str to data
    try:
        for col in ['2014', '2015', '2016', '2017', '2018', '2019', '2020',]:
            elec_data[col] = elec_data[col].str.strip().str.replace(',','')
            elec_data[col] = elec_data[col].astype(float)
    except:
        pass
    # rename parish col
    elec_data['Parish'] = elec_data['PARISH']
    elec_data = elec_data.drop(['PARISH'],axis=1)
    # rename parish names
    elec_data.Parish = elec_data.Parish.str.replace('Portmore', 'St. Catherine')
    elec_data.Parish = elec_data.Parish.str.replace('KSAN', 'St. Andrew')
    elec_data.Parish = elec_data.Parish.str.replace('KSAS', 'Kingston')
    # group KSAN and KSAS data
    elec_data = elec_data.groupby(by=['Parish']).sum().reset_index()
    # get latest year
    elec_data = elec_data[['Parish',max(elec_data.columns.drop('Parish'))]]
    return elec_data


def append_energy_data_to_boundaries(boundaries,elec_data):
    '''Combine energy data and admin boundaries
    '''
    elec_dict = elec_data.set_index('Parish')['2020'].to_dict()
    boundaries['consumption'] = boundaries['PARISH'].map(elec_dict)
    boundaries['ei'] = boundaries.consumption / boundaries.population
    boundaries['ei_uom'] = 'KW/person'
    return boundaries[['PARISH','population','consumption','ei','ei_uom',]]


def get_ei_by_parish():
    '''Get a dataframe of electricity intensities by parish
    '''
    elec_data = process_raw_consumption_data('../data/energy-demand/energy_consumption.csv')
    boundaries = gpd.read_file('../data/incoming_data/admin_boundaries.gpkg',layer='admin1')
    boundaries = append_energy_data_to_boundaries(boundaries,elec_data)
    return boundaries

In [ ]:
elec_data = process_raw_consumption_data('../data/energy-demand/energy_consumption.csv')
elec_data

In [ ]:
boundaries = gpd.read_file('../data/incoming_data/admin_boundaries.gpkg',layer='admin1')
boundaries

In [ ]:
boundaries = gpd.read_file('../data/population-russell/population.gpkg')
boundaries.PARISH = boundaries.PARISH.str.title()
boundaries.loc[boundaries.PARISH.str.contains('Catherine'),'PARISH'] = 'St. Catherine'
boundaries.loc[boundaries.PARISH.str.contains('Mary'),'PARISH'] = 'St. Mary'
boundaries.loc[boundaries.PARISH.str.contains('Andrew'),'PARISH'] = 'St. Andrew'
boundaries.loc[boundaries.PARISH.str.contains('Elizabeth'),'PARISH'] = 'St. Elizabeth'
boundaries.loc[boundaries.PARISH.str.contains('Thomas'),'PARISH'] = 'St. Thomas'
boundaries.loc[boundaries.PARISH.str.contains('James'),'PARISH'] = 'St. James'
boundaries.loc[boundaries.PARISH.str.contains('Ann'),'PARISH'] = 'St. Ann'
boundaries = boundaries.groupby(by='PARISH').sum().reset_index()
boundaries

In [ ]:
append_energy_data_to_boundaries(boundaries,elec_data)

In [ ]:
6600/8760